# Download ERA5 data 

In [1]:
import os, sys
from netCDF4 import Dataset
import numpy as np
import zipfile 
import plotly.express as px
#import xarray as xr
sys.path.append('../src/')
from Biologging_Toolkit.auxiliary import Api_ERA
from Biologging_Toolkit.utils import *

c:\Users\barsro\Documents\SES_tags\notebooks\../src\Biologging_Toolkit


## Pre-requirements


In order to download ERA5 data using an API, you will have to get a UDI and a personal key. To get those, you will have to register to the Copernicus platform using the following link : https://cds.climate.copernicus.eu/user/login
<br>Enter/copy paste your UDI and your key in the cells below

In [2]:
personal_access_token = '5687bb83-bb46-4334-8925-930439a13112'

Below you can add a filename after your variable names (eg, `filename = 'ERA5'` will give you __u10_ERA5__).
<br>
By default, files will be downloaded in your home directory, in the subdirectory __api__ , but you can change it through the variable __path__ (otherwise let default value `path = 'None'`).

In [3]:
filename = 'ml19_292b'
path = '.'

## Variable, time period and geographical location

ERA5 provides hourly estimates for a large number of ocean-wave and atmospheric variables (see list [here](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)). Variable names must be formatted with lowercase first letters, and composed words spaced by underscores (eg, Sea surface temperature in `'sea_surface_temperature'`, 10m u-component of wind in `'10m_u_component_of_wind'`). Use the alias `'all'` to select all available months, days or hours, otherwise set one or several elements as a list (eg, `'days = ['01','02']'`).

### You can get boundaries from built dataset

In [4]:
# data_path = '/home6/grosmaan/Documents/data/ml17_280a_sens.nc'

data_path = 'e:/individus_brut/ml19_292b/ml19_292b_sens.nc'
get_boundaries_metadata(data_path)

Dataset begins at  2019-10-19 06:00:00
and ends at        2019-12-27 03:00:00
The GPS boundaries are :
    - South  : -43.00°
    - North  : -38.75°
    - West   : -65.00°
    - East   : -49.50°


In [5]:
# CHOOSE WHICH SINGLE LEVELS TO DOWNLOAD IN data
variables = ['10m_u_component_of_wind', '10m_v_component_of_wind', 'total_precipitation'] 

years = ['2017'] 

months =  ['10','11']
#Or set one or several months as a list
# ['01','02','03','04','05','06','07','08','09','10','11','12']

days = 'all'
#Or set one or several days as a list
# ['01', '02', 03','04','05','06','07','08','09','10','11','12','13','14','15','16','17',
                   #'18','19','20','21','22','23','24','25','26','27','28','29','30','31']

hours = 'all'
#Or set one or several hours as a list
#['00:00','01:00','02:00','03:00','04:00', '05:00','06:00','07:00','08:00',
                     #'09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00',
                     #'20:00','21:00','22:00','23:00']

# Boundaries of the geographical area in decimal degrees
south_boundary = -48
north_boundary = -42
west_boundary = 51.25
east_boundary = 68.5


## Print summary and run download

In [6]:
Api_ERA.make_cds_file(personal_access_token, path)

In [7]:
Api_ERA.return_cdsbeta(filename, variables, years, months, days, hours,  [north_boundary, west_boundary, south_boundary, east_boundary])

You have selected : 

   - 10m_u_component_of_wind
   - 10m_v_component_of_wind
   - total_precipitation

for the following times
Years : ['2017'] 
 Months : ['10', '11'] 
 Days : all 
 Hours : all

Your boundaries are : North -42°, South -48°, East 68.5°, West 51.25°


c:\Users\barsro\.conda\envs\rain\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds-beta.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\barsro\.conda\envs\rain\Lib\site-packages\datapi\api_client.py:83: UserWarning: 404 Client Error: Not Found for url: https://cds-beta.climate.copernicus.eu/api/catalogue/v1/messages
  warnings.warn(str(exc), UserWarning)
c:\Users\barsro\.conda\envs\rain\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds-beta.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


HTTPError: 404 Client Error: Not Found for url: https://cds-beta.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels

### Code to visualize your variables

In [ ]:
variable_name= 'v10'

with zipfile.ZipFile(os.path.join(path,filename+'.zip'), 'r') as zip_ref:
    zip_ref.extractall(path)

try :
    nc = Dataset(os.path.join(path,'data_stream-oper.nc'), 'r')
    ds = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))
except FileNotFoundError :  
    print('no ERA data with this variable')
    sys.exit()

ds = ds[variable_name]

fig = px.imshow(ds, animation_frame='valid_time', zmin=float(ds.min()), zmax=float(ds.max()), color_continuous_scale='RdBu_r')

fig.show()